In [2]:
import sys, subprocess
# System operations
import os

# Type hinting
from typing import Any, List, Mapping, Tuple, Union

# Data visualization
import matplotlib.pyplot as plt

# Data manipulation
import numpy as np
import pandas as pd
import gymnasium as gym
import torch as th

# CityLearn
import citylearn
from citylearn.data import DataSet
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper
from citylearn.citylearn import CityLearnEnv

# Baseline RL algorithms
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.monitor import Monitor

# Utils
from utils.env_utils import DiscretizeActionWrapper, keep_only_electrical_storage, keep_only_core_observations

print('Python:', sys.version.split()[0])
print('Torch :', th.__version__)
print('CityLearn:', citylearn.__version__)
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "optuna", "optuna-dashboard"])
print("Optuna installed.")

2025-08-27 16:42:00.434513: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-27 16:42:00.458205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756305720.486376  341288 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756305720.494615  341288 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756305720.516043  341288 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Python: 3.11.7
Torch : 2.7.0+cu126
CityLearn: 2.4.1
Optuna installed.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /hkfs/home/haicore/iai/cj9272/citylearn_env/bin/python -m pip install --upgrade pip


In [3]:
DATASET_NAME = 'citylearn_challenge_2023_phase_3_1'  # adjust if needed
ROOT_DIR = r'/hkfs/home/haicore/iai/cj9272/Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1'
REWARD_FN = {  # CostReward 
    'type': 'citylearn.reward_function.CostReward',
    'attributes': {}
}
PRICING_FILE = 'pricing_germany_2023_june_to_august.csv'
ACTION_LABELS = ['100%_discharge','50%_discharge','idle','50%_charge','100%_charge']
NAME_TO_FRAC = {
    '100%_discharge': -1.0,
    '50%_discharge': -0.5,
    'idle': 0.0,
    '50%_charge': 0.5,
    '100%_charge': 1.0,
}
EVAL_EPISODES = 10

In [4]:
observations = [
    # building_df
    'month', 'hour', 'day_type', 'daylight_savings_status',
    'indoor_dry_bulb_temperature',
    'average_unmet_cooling_setpoint_difference',
    'indoor_relative_humidity',
    'non_shiftable_load', 'dhw_demand',
    'cooling_demand', 'heating_demand',
    'solar_generation', 'occupant_count',
    'indoor_dry_bulb_temperature_cooling_set_point',
    'indoor_dry_bulb_temperature_heating_set_point', 'hvac_mode',
    # weather_df
    'outdoor_dry_bulb_temperature',
    'outdoor_relative_humidity',
    'diffuse_solar_irradiance',
    'direct_solar_irradiance',
    'outdoor_dry_bulb_temperature_predicted_1',
    'outdoor_dry_bulb_temperature_predicted_2',
    'outdoor_dry_bulb_temperature_predicted_3',
    'outdoor_relative_humidity_predicted_1',
    'outdoor_relative_humidity_predicted_2',
    'outdoor_relative_humidity_predicted_3',
    'diffuse_solar_irradiance_predicted_1',
    'diffuse_solar_irradiance_predicted_2',
    'diffuse_solar_irradiance_predicted_3',
    'direct_solar_irradiance_predicted_1',
    'direct_solar_irradiance_predicted_2',
    'direct_solar_irradiance_predicted_3',
    # carbon_df 
    'carbon_intensity',
    # pricing_df
    'electricity_pricing',
    'electricity_pricing_predicted_1',
    'electricity_pricing_predicted_2',
    'electricity_pricing_predicted_3'
]
bld_cols = [
            'month', 'hour', 'day_type', 'daylight_savings_status',
            'indoor_dry_bulb_temperature',
            'average_unmet_cooling_setpoint_difference',
            'indoor_relative_humidity', 'non_shiftable_load',
            'dhw_demand', 'cooling_demand', 'heating_demand',
            'solar_generation', 'occupant_count',
            'indoor_dry_bulb_temperature_cooling_set_point',
            'indoor_dry_bulb_temperature_heating_set_point',
            'hvac_mode'
]
wth_cols = [
    'outdoor_dry_bulb_temperature',
    'outdoor_relative_humidity',
    'diffuse_solar_irradiance',
    'direct_solar_irradiance'
]

In [5]:
# --- Load schema ---
dataset = DataSet()
schema = dataset.get_schema(DATASET_NAME)
schema['root_directory'] = ROOT_DIR
schema['reward_function'] = REWARD_FN

price_file = PRICING_FILE # Set pricing file
if 'buildings' not in schema:
    raise RuntimeError("schema does not contain 'buildings' (make sure schema is loaded first)")
for bname, bconf in schema['buildings'].items():
    bconf['pricing'] = price_file

schema = keep_only_electrical_storage(schema) # Activate only the electrical storage control (fix "Expected 18 actions but got 1")
schema = keep_only_core_observations(schema, extra_keep=['carbon_intensity', 'non_shiftable_load'], drop_predictions=False)


INFO:root:Go here /home/iai/cj9272/.cache/citylearn/v2.4.1/datasets/citylearn_challenge_2023_phase_3_1/schema.json 


In [6]:
class TrainLoggerCallback(BaseCallback):
    """Logging State, Action, Reward per step and Loss per update phase."""
    def __init__(self, verbose=0):
        super().__init__(verbose)
        # Will collect a dict per env-step
        self.rows = []
        # Loss values and their timesteps (global)
        self.losses = []
        self.loss_timesteps = []
        # Completed episode returns (global list)
        self.episode_rewards = []
        # Placeholders for per-env tracking
        self._current_ep_rewards = []         # sum of rewards in current episode per env
        self._current_ep_counts = []          # episode index per env
        self._current_step_in_episode = []    # step counter (0..T-1) per env

        # DataFrames to populate at end
        self.df = pd.DataFrame()
        self.ep_df = pd.DataFrame()

    def _on_training_start(self) -> None:
        try:
            n_envs = self.training_env.num_envs
        except AttributeError:
            n_envs = 1
        # initialize counters per sub-env
        self._current_ep_rewards = [0.0] * n_envs
        self._current_ep_counts = [1] * n_envs
        self._current_step_in_episode = [0] * n_envs
        super()._on_training_start()

    def _on_step(self) -> bool:
        obs_vec = self.locals.get("new_obs")
        acts    = self.locals.get("actions")
        rews    = self.locals.get("rewards")
        dones   = self.locals.get("dones")
        step    = int(self.num_timesteps)

        # log loss if present
        loss_val = self.logger.name_to_value.get("train/loss")
        if loss_val is not None:
            self.losses.append(float(loss_val))
            self.loss_timesteps.append(step)

        # iterate each sub-env
        for idx, (obs, act, rew, done) in enumerate(zip(obs_vec, acts, rews, dones)):
            # flatten observation
            flat = obs.flatten().tolist()
            # build row with metadata
            row = {f"x{i}": flat[i] for i in range(len(flat))}
            row.update({
                "env_id": idx,
                "episode": self._current_ep_counts[idx],
                "step_in_ep": self._current_step_in_episode[idx],
                "action": int(act),
                "reward": float(rew),
                "global_step": step
            })
            row['action_name'] = ACTION_LABELS[int(act)]
            row['action_frac'] = NAME_TO_FRAC[row['action_name']]
            self.rows.append(row)

            # accumulate per-episode reward
            self._current_ep_rewards[idx] += float(rew)
            # increment step in episode
            self._current_step_in_episode[idx] += 1

            # if end of episode for this env
            if done:
                # log reward and finalize episode
                print(f"Env {idx} Episode {self._current_ep_counts[idx]} done at global step {step}, total reward: {self._current_ep_rewards[idx]:.3f}")
                self.episode_rewards.append(self._current_ep_rewards[idx])
                # reset for next episode
                self._current_ep_rewards[idx] = 0.0
                self._current_ep_counts[idx] += 1
                self._current_step_in_episode[idx] = 0

        return True

    def _on_training_end(self) -> None:
        # build full-step DataFrame
        self.df = pd.DataFrame(self.rows)
        # build episodes summary DataFrame
        self.ep_df = pd.DataFrame({
            "episode_global": range(1, len(self.episode_rewards) + 1),
            "return": self.episode_rewards
        })
        super()._on_training_end()

In [7]:
import copy, os, time, json, pickle
import numpy as np
import pandas as pd
from citylearn.citylearn import CityLearnEnv
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper

# Keep only one building: 1-D action
one_building = "Building_1"
schema_1b = dict(schema)  # shallow copy
schema_1b['buildings'] = {k: v for k, v in schema['buildings'].items() if k == one_building}
if not schema_1b['buildings']:
    raise RuntimeError(f"{one_building} not found in schema['buildings']")

train_env = CityLearnEnv(schema_1b, central_agent=True)
train_env = NormalizedObservationWrapper(train_env)
train_env = StableBaselines3Wrapper(train_env)
# Discretize action space for DQN:
train_env = DiscretizeActionWrapper(train_env, n_bins=5)
train_env = Monitor(train_env)

train_callback = TrainLoggerCallback()

T = train_env.unwrapped.time_steps  # 2208
num_episodes = 10
TOTAL_TIMESTEPS = num_episodes * T

model = DQN(
    policy='MlpPolicy',
    env=train_env,
    seed=0,
    #learning_starts=1000,
    #learning_rate=3e-4,
    verbose=1, # logging: info
    policy_kwargs=dict(net_arch=[256, 256, 256], activation_fn=th.nn.ReLU),
)

/hkfs/home/haicore/iai/cj9272/Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1
Dataset '/hkfs/home/haicore/iai/cj9272/Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1' copied to '/hkfs/home/haicore/iai/cj9272/Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1/../../../../results/2025-08-27_16-42-07'
Using cuda device
Wrapping the env in a DummyVecEnv.


In [ ]:
start_time = time.time()
model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=train_callback)
print(f"Total time elapsed for {TOTAL_TIMESTEPS} steps: {(time.time()-start_time):.2f} seconds")
model.save("dqn_01")
try:
    model.save_replay_buffer("dqn_01_replaybuffer.pkl")
except Exception as e:
    print("Replay buffer not saved:", e)
print("---------------Train callback: \n", train_callback.df)
print("---------------Episode rewards: \n", train_callback.ep_df)
# Save training logs
train_callback.df.to_csv("train_steps.csv", index=False)
train_callback.ep_df.to_csv("train_episodes.csv", index=False)

Env 0 Episode 1 done at global step 2207, total reward: -287.480
Env 0 Episode 2 done at global step 4414, total reward: -266.891
Env 0 Episode 3 done at global step 6621, total reward: -266.357


In [ ]:
schema_eval = copy.deepcopy(schema_1b)
eval_env = CityLearnEnv(schema_eval, central_agent=True)
eval_env = NormalizedObservationWrapper(eval_env)
eval_env = StableBaselines3Wrapper(eval_env)
eval_env = DiscretizeActionWrapper(eval_env, n_bins=5)

model = DQN.load("dqn_01", env=eval_env)

/hkfs/home/haicore/iai/cj9272/Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1
Dataset '/hkfs/home/haicore/iai/cj9272/Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1' copied to '/hkfs/home/haicore/iai/cj9272/Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1/../../../../results/2025-08-27_13-03-06'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
# Eval: collect discrete actions and rewards per step (overwrites simple eval_dqn)
def eval_dqn(model, eval_env, schema, episodes=3, deterministic=True):
    """Deterministic evaluation that returns actions/rewards for plotting."""
    ep_lengths, actions_disc_list, step_rewards_all = [], [], []
    for _ in range(episodes):
        obs, _ = eval_env.reset()
        done = False
        ep_actions, ep_rewards = [], []
        while not done:
            act_disc, _ = model.predict(obs, deterministic=deterministic)
            obs, r, terminated, truncated, _ = eval_env.step(act_disc)
            done = bool(terminated or truncated)
            ep_actions.append(int(act_disc))
            ep_rewards.append(float(r))
        ep_lengths.append(len(ep_rewards))
        actions_disc_list.append(np.array(ep_actions, dtype=int))
        step_rewards_all.extend(ep_rewards)

    # Pricing for Building_1 
    building_name = 'Building_1'
    price_file = schema['buildings'][building_name]['pricing']
    price_path = os.path.join(ROOT_DIR, price_file)
    prc = pd.read_csv(price_path)
    T = len(prc)

    return {
        'ep_lengths': np.array(ep_lengths, dtype=int),
        'actions_disc_list': actions_disc_list,
        'step_rewards': np.array(step_rewards_all, dtype=float),
        'price_df': prc,
        'episode_len': T,
        'building_name': building_name,
    }

# Run eval and store results
eval_results = eval_dqn(model, eval_env, schema_eval, episodes=10, deterministic=True)
print('Eval episodes:', len(eval_results['ep_lengths']), 'total steps:', len(eval_results['step_rewards']))

# Save eval results
eval_path = "eval_results.pkl"
with open(eval_path, "wb") as f:
    pickle.dump(eval_results, f)

Eval episodes: 10 total steps: 22070


In [ ]:
# Optuna objective for DQN on CityLearn (single building, same wrappers)
import os, copy, json
import numpy as np
import optuna
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor

# Reuse your DiscretizeActionWrapper, NormalizedObservationWrapper, StableBaselines3Wrapper, CityLearnEnv
# Assumes 'schema', 'ROOT_DIR', 'one_building' already exist from your previous cells.

ART_DIR = os.path.join(os.getcwd(), "artifacts")
os.makedirs(ART_DIR, exist_ok=True)
STORAGE_URL = f"sqlite:///{os.path.join(ART_DIR, 'optuna_dqn.db')}"
STUDY_NAME  = "dqn_citylearn"

def make_env_one_building(schema_src, building_name="Building_1", n_bins=5, monitor=False):
    schema_1b = copy.deepcopy(schema_src)
    schema_1b['buildings'] = {k: v for k, v in schema_src['buildings'].items() if k == building_name}
    if not schema_1b['buildings']:
        raise RuntimeError(f"{building_name} not found in schema['buildings']")
    env = CityLearnEnv(schema_1b, central_agent=True)
    env = NormalizedObservationWrapper(env)
    env = StableBaselines3Wrapper(env)
    env = DiscretizeActionWrapper(env, n_bins=n_bins)
    if monitor:
        env = Monitor(env)
    return env

def evaluate_mean_reward(model, env, episodes=2, deterministic=True):
    rewards = []
    for _ in range(episodes):
        obs, _ = env.reset()
        done = False
        ep_ret = 0.0
        while not done:
            act, _ = model.predict(obs, deterministic=deterministic)
            obs, r, terminated, truncated, _ = env.step(act)
            ep_ret += float(r)
            done = bool(terminated or truncated)
        rewards.append(ep_ret)
    return float(np.mean(rewards))

class OptunaEvalPruningCallback:
    def __init__(self, trial, model, eval_env, eval_every_steps, eval_episodes=2):
        self.trial = trial
        self.model = model
        self.eval_env = eval_env
        self.eval_every_steps = int(eval_every_steps)
        self.eval_episodes = int(eval_episodes)
        self._last_step = 0
        self.is_pruned = False

    def __call__(self, locals_, globals_):
        # Called by SB3 when using callback= in learn (old-style callable)
        step = int(locals_.get("self").num_timesteps)
        if step - self._last_step >= self.eval_every_steps:
            self._last_step = step
            mean_r = evaluate_mean_reward(self.model, self.eval_env, episodes=self.eval_episodes, deterministic=True)
            self.trial.report(mean_r, step=step)
            if self.trial.should_prune():
                self.is_pruned = True
                return False  # stop training
        return True

def objective(trial: optuna.Trial) -> float:
    # Search space
    lr      = trial.suggest_float("learning_rate", 1e-5, 3e-3, log=True)
    gamma   = trial.suggest_float("gamma", 0.90, 0.9999)
    buffer  = trial.suggest_int("buffer_size", 50_000, 200_000, step=25_000)
    batch   = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    tau     = trial.suggest_float("tau", 0.005, 1.0, log=True)
    tgt_upd = trial.suggest_int("target_update_interval", 250, 5000, step=250)
    train_f = trial.suggest_int("train_freq", 1, 8)
    grad_st = trial.suggest_int("gradient_steps", 1, 4)
    expl_fr = trial.suggest_float("exploration_fraction", 0.05, 0.4)
    expl_fin= trial.suggest_float("exploration_final_eps", 0.01, 0.1)
    starts  = trial.suggest_int("learning_starts", 500, 5000, step=500)
    arch    = trial.suggest_categorical("net_arch", [(256,256), (256,256,256), (512,512)])

    # Build train/eval envs
    train_env = make_env_one_building(schema, building_name="Building_1", n_bins=5, monitor=True)
    eval_env  = make_env_one_building(schema, building_name="Building_1", n_bins=5, monitor=False)
    T = train_env.unwrapped.time_steps

    # Episodes per trial (keep small for speed)
    episodes_per_trial = 3
    total_steps = episodes_per_trial * T

    model = DQN(
        policy="MlpPolicy",
        env=train_env,
        seed=0,
        learning_rate=lr,
        gamma=gamma,
        buffer_size=buffer,
        batch_size=batch,
        tau=tau,
        target_update_interval=tgt_upd,
        train_freq=train_f,
        gradient_steps=grad_st,
        exploration_fraction=expl_fr,
        exploration_final_eps=expl_fin,
        learning_starts=starts,
        verbose=0,
        policy_kwargs=dict(net_arch=list(arch), activation_fn=th.nn.ReLU),
        device="auto",
    )

    # Evaluation + pruning during training
    cb = OptunaEvalPruningCallback(trial, model, eval_env, eval_every_steps=T//2, eval_episodes=2)
    try:
        model.learn(total_timesteps=total_steps, callback=cb)
        if cb.is_pruned:
            raise optuna.TrialPruned()
    finally:
        # Ensure envs are closed
        try: train_env.close()
        except: pass
        try: eval_env.close()
        except: pass

    # Final score
    eval_env = make_env_one_building(schema, building_name="Building_1", n_bins=5, monitor=False)
    score = evaluate_mean_reward(model, eval_env, episodes=3, deterministic=True)
    eval_env.close()

    # Save best-so-far model snapshot
    trial.set_user_attr("final_mean_reward", score)
    return score

# Create/continue a study
study = optuna.create_study(
    study_name=STUDY_NAME,
    direction="maximize",
    storage=STORAGE_URL,
    load_if_exists=True,
    sampler=optuna.samplers.TPESampler(seed=0),
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=1), # NopPruner()
)
print("Study ready:", STUDY_NAME, STORAGE_URL)

[I 2025-08-27 13:23:07,751] Using an existing study with name 'dqn_citylearn' instead of creating a new one.


Study ready: dqn_citylearn sqlite:////hkfs/home/haicore/iai/cj9272/artifacts/optuna_dqn.db


In [ ]:
# Optuna hyperparameter optimization

N_TRIALS = 3  # increase as needed
#study.optimize(objective, n_trials=N_TRIALS, gc_after_trial=True)
#print("Done. Trials now:", len(study.trials))

In [ ]:
# 6 DQN plots matching the PPO ones

import os, copy, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gymnasium as gym
from citylearn.citylearn import CityLearnEnv

# Ensure INT_TO_FRAC and ACTION_LABELS exist
INT_TO_FRAC = np.array([-1.0, -0.5, 0.0, 0.5, 1.0], dtype=np.float32)
ACTION_LABELS = ['100%_discharge','50%_discharge','idle','50%_charge','100%_charge']

# ------------- Helpers -------------
def unwrap_to_citylearn(env):
    cur, seen = env, set()
    while cur is not None and id(cur) not in seen:
        seen.add(id(cur))
        if isinstance(cur, CityLearnEnv):
            return cur
        cur = getattr(cur, "env", getattr(cur, "unwrapped", None))
    raise RuntimeError("CityLearnEnv not found inside wrappers.")

def capture_ep_series_dqn(model, env, building_name="Building_1", deterministic=True, max_steps=None):
    """Run one episode (or first max_steps) and capture per-step action, reward, and base env series."""
    base = unwrap_to_citylearn(env)

    # Building index
    bld_names = [b.name for b in base.buildings]
    if building_name not in bld_names:
        building_name = bld_names[0]
    b_idx = bld_names.index(building_name)

    # Observation indices
    obs_names = getattr(base, 'observation_names', None)
    if obs_names is None:
        raise AttributeError("CityLearnEnv has no 'observation_names'.")
    obs_b = obs_names[b_idx]
    name_to_idx = {n: i for i, n in enumerate(obs_b)}

    i_net = name_to_idx.get('net_electricity_consumption')
    if i_net is None:
        i_net = name_to_idx.get('net_electricity_consumption_without_storage')
    i_nsl = name_to_idx.get('non_shiftable_load')
    i_price = name_to_idx.get('electricity_pricing')
    i_solar = name_to_idx.get('solar_generation')

    # Run rollout
    obs, _ = env.reset()
    done, steps = False, 0
    t, a_id_list, a_frac_list, rew_list = [], [], [], []
    net_true, nsl_true, price_true, solar_true = [], [], [], []

    while not done and (max_steps is None or steps < max_steps):
        act, _ = model.predict(obs, deterministic=deterministic)

        # Discrete action id -> fraction
        if isinstance(env.action_space, gym.spaces.Discrete):
            a_id = int(act)
        elif isinstance(env.action_space, gym.spaces.MultiDiscrete):
            a_id = int(np.asarray(act).reshape(-1)[b_idx])
        else:
            val = float(np.asarray(act).reshape(-1)[b_idx])
            a_id = int(np.argmin(np.abs(INT_TO_FRAC - val)))
        a_frac = float(INT_TO_FRAC[a_id])

        obs, r, terminated, truncated, _ = env.step(act)
        done = bool(terminated or truncated)

        raw = base.observations[b_idx]
        t.append(steps)
        a_id_list.append(a_id)
        a_frac_list.append(a_frac)
        rew_list.append(float(r))
        net_true.append(float(raw[i_net]) if i_net is not None else np.nan)
        nsl_true.append(float(raw[i_nsl]) if i_nsl is not None else np.nan)
        price_true.append(float(raw[i_price]) if i_price is not None else np.nan)
        solar_true.append(float(raw[i_solar]) if i_solar is not None else np.nan)

        steps += 1

    df = pd.DataFrame({
        't': np.arange(len(t)),
        'action_id': a_id_list,
        'action_frac': a_frac_list,
        'reward': rew_list,
        'net_load': net_true,
        'non_shiftable_load': nsl_true,
        'price': price_true,
        'solar_generation': solar_true,
    })
    return df, building_name

# ------------- Plot 1 + 2: Actions over time + distribution -------------
# Use cached eval_results if available, else run a quick eval
def _ensure_eval_results():
    global eval_results
    if 'eval_results' in globals():
        return eval_results
    # Fallback quick eval (deterministic)
    def eval_dqn_quick(model, eval_env, schema, episodes=3, deterministic=True):
        ep_lengths, actions_disc_list, step_rewards_all = [], [], []
        for _ in range(episodes):
            obs, _ = eval_env.reset()
            done = False
            ep_actions, ep_rewards = [], []
            while not done:
                act_disc, _ = model.predict(obs, deterministic=deterministic)
                obs, r, terminated, truncated, _ = eval_env.step(act_disc)
                done = bool(terminated or truncated)
                ep_actions.append(int(act_disc))
                ep_rewards.append(float(r))
            ep_lengths.append(len(ep_rewards))
            actions_disc_list.append(np.array(ep_actions, dtype=int))
            step_rewards_all.extend(ep_rewards)
        # Pricing for Building_1
        bname = 'Building_1'
        price_file = schema['buildings'][bname]['pricing']
        price_path = os.path.join(ROOT_DIR, price_file)
        prc = pd.read_csv(price_path)
        return {
            'ep_lengths': np.array(ep_lengths, dtype=int),
            'actions_disc_list': actions_disc_list,
            'step_rewards': np.array(step_rewards_all, dtype=float),
            'price_df': prc,
            'episode_len': len(prc),
            'building_name': bname,
        }

    schema_eval_local = copy.deepcopy(schema_1b) if 'schema_1b' in globals() else copy.deepcopy(schema)
    eval_env_local = CityLearnEnv(schema_eval_local, central_agent=True)
    eval_env_local = NormalizedObservationWrapper(eval_env_local)
    eval_env_local = StableBaselines3Wrapper(eval_env_local)
    eval_env_local = DiscretizeActionWrapper(eval_env_local, n_bins=5)

    eval_results = eval_dqn_quick(model, eval_env_local, schema_eval_local, episodes=5, deterministic=True)
    return eval_results

er = _ensure_eval_results()
actions_disc = np.concatenate(er['actions_disc_list'], axis=0)
actions_frac = pd.Series(actions_disc).map({0:-1.0, 1:-0.5, 2:0.0, 3:0.5, 4:1.0}).to_numpy()
hour_index = np.arange(len(actions_frac))
building_name = er['building_name']

# Plot 1: Actions over time (raw + MA)
window = 500
act_ma = pd.Series(actions_frac).rolling(window, min_periods=1).mean().to_numpy()
plt.figure(figsize=(7,3))
plt.scatter(hour_index, actions_frac, s=5, alpha=0.35, label='raw')
plt.plot(hour_index, act_ma, color='C2', lw=2, label=f'ma {window}')
plt.title(f'DQN Eval Actions over time (Raw + MA) - {building_name}')
plt.xlabel('Timestep'); plt.ylabel('Action [-1..1]'); plt.grid(True); plt.legend(loc='upper right')
plt.yticks(INT_TO_FRAC.tolist(), ACTION_LABELS)
plt.tight_layout(); plt.show()

# Plot 2: Action Distribution (5-bin)
counts = pd.Series(actions_disc).value_counts().reindex(range(5)).fillna(0).astype(int)
plt.figure(figsize=(4,3))
ax = counts.plot(kind='bar', color=['C0','C1','C2','C3','C4'])
ax.set_xticklabels(ACTION_LABELS, rotation=45, ha='right')
total = counts.sum()
ax.bar_label(ax.containers[0], labels=[f'{int(v)}\n({v/total:.1%})' for v in counts.values])
ax.set_ylim(0, counts.max()*1.5)
plt.ylabel('Count'); plt.title('Action Distribution'); plt.tight_layout(); plt.show()

# ------------- Plot 3: First 20 steps — actions vs loads -------------
# ------------- Plots 4,5,6: Reward vs Price/Solar/Net (episode 1) -------------

def plot_first_30_steps_actions_vs_loads_dqn(model, env, building_name="Building_1", deterministic=True):
    df30, bname = capture_ep_series_dqn(model, env, building_name, deterministic=deterministic, max_steps=30)

    t = df30['t'].to_numpy()
    net = df30['net_load'].to_numpy()
    nsl = df30['non_shiftable_load'].to_numpy()
    act = df30['action_frac'].to_numpy()

    fig, ax1 = plt.subplots(figsize=(8, 3))
    ax1.plot(t, net, color='C4', marker='o', label='Net Load (with storage)')
    ax1.plot(t, nsl, color='C1', ls='--', marker='x', label='Non-shiftable Load (baseline)')
    ax1.set_xlabel('Timestep')
    ax1.set_xticks(t[::2])
    ax1.set_ylabel('Load')
    ax1.grid(True, alpha=0.3)
    ax1.legend(loc='upper left', frameon=False)

    ax2 = ax1.twinx()
    ax2.step(t, act, where='mid', color='C0', label='Action')
    ax2.set_ylabel('Action')
    ax2.set_ylim(-1.1, 1.1)
    ax2.set_yticks(INT_TO_FRAC.tolist())
    ax2.set_yticklabels(ACTION_LABELS)

    plt.title(f'DQN — first 30 steps — {bname} (deterministic={deterministic})')
    plt.tight_layout()
    plt.show()

def dqn_ep1_reward_vs_series(model, env, building_name="Building_1", deterministic=True):
    df, bname = capture_ep_series_dqn(model, env, building_name, deterministic=deterministic, max_steps=None)

    window_reward = 200
    window_x = 24
    t = df['t'].to_numpy()
    reward_ma = pd.Series(df['reward']).rolling(window_reward, min_periods=1).mean().to_numpy()
    price_ma  = pd.Series(df['price']).rolling(window_x, min_periods=1).mean().to_numpy()
    solar_ma  = pd.Series(df['solar_generation']).rolling(window_x, min_periods=1).mean().to_numpy()
    net_ma    = pd.Series(df['net_load']).rolling(window_x, min_periods=1).mean().to_numpy()


    # 4) Reward vs Net Load
    fig, ax1 = plt.subplots(figsize=(12, 3.5))
    ax1.plot(t, reward_ma, color='C3', label=f'Reward MA (w={window_reward})')
    ax1.set_xlabel('Timestep'); ax1.set_ylabel(f'Reward MA ({window_reward})', color='C3')
    ax1.tick_params(axis='y', labelcolor='C3'); ax1.grid(True, axis='x', alpha=0.3)
    ax2 = ax1.twinx()
    ax2.plot(t, net_ma, color='C4', label=f'Net Load MA (w={window_x})')
    ax2.set_ylabel('Net Electricity Consumption', color='C4'); ax2.tick_params(axis='y', labelcolor='C4')
    lines = ax1.get_lines() + ax2.get_lines()
    ax1.legend(lines, [l.get_label() for l in lines], loc='upper right', frameon=False)
    ax1.set_title(f'DQN Episode 1 — Reward vs Net Load — {bname}')
    plt.tight_layout(); plt.show()


    # 5) Reward vs Electricity Price
    fig, ax1 = plt.subplots(figsize=(12, 3.5))
    ax1.plot(t, reward_ma, color='C3', label=f'Reward MA (w={window_reward})')
    ax1.set_xlabel('Timestep'); ax1.set_ylabel(f'Reward MA ({window_reward})', color='C3')
    ax1.tick_params(axis='y', labelcolor='C3'); ax1.grid(True, axis='x', alpha=0.3)
    ax2 = ax1.twinx()
    ax2.plot(t, price_ma, color='C0', label=f'Price MA (w={window_x})')
    ax2.set_ylabel('Electricity Price', color='C0'); ax2.tick_params(axis='y', labelcolor='C0')
    lines = ax1.get_lines() + ax2.get_lines()
    ax1.legend(lines, [l.get_label() for l in lines], loc='upper right', frameon=False)
    ax1.set_title(f'DQN Episode 1 — Reward vs Electricity Price — {bname}')
    plt.tight_layout(); plt.show()

    # 6) Reward vs Solar Generation
    fig, ax1 = plt.subplots(figsize=(12, 3.5))
    ax1.plot(t, reward_ma, color='C3', label=f'Reward MA (w={window_reward})')
    ax1.set_xlabel('Timestep'); ax1.set_ylabel(f'Reward MA ({window_reward})', color='C3')
    ax1.tick_params(axis='y', labelcolor='C3'); ax1.grid(True, axis='x', alpha=0.3)
    ax2 = ax1.twinx()
    ax2.plot(t, solar_ma, color='C2', label=f'Solar Gen MA (w={window_x})')
    ax2.set_ylabel('Solar Generation', color='C2'); ax2.tick_params(axis='y', labelcolor='C2')
    lines = ax1.get_lines() + ax2.get_lines()
    ax1.legend(lines, [l.get_label() for l in lines], loc='upper right', frameon=False)
    ax1.set_title(f'DQN Episode 1 — Reward vs Solar Generation — {bname}')
    plt.tight_layout(); plt.show()

plot_first_30_steps_actions_vs_loads_dqn(model, eval_env, building_name="Building_1", deterministic=False)
dqn_ep1_reward_vs_series(model, eval_env, building_name="Building_1", deterministic=False)

Couldn't import dot_parser, loading of dot files will not be possible.


NameError: name 'schema' is not defined